# MultiQC Jupyter Notebook Example

This notebook has some example code showing how MultiQC can be used within an interactive analysis, such as a Jupyter Notebook.

MultiQC is written in Python, so must be used in a Python environment. If possible, Python 3.5 or later is recommended. MultiQC can be installed in a variety of ways: see [the documentation](https://multiqc.info/docs/#installing-multiqc) for more information. Note that MultiQC must be installed into the notebook _kernal_.

First, let's install MultiQC using pip (note the `%` magic which installs the package into the kernel and not the jupyter environment).

> NB: Support for imports is only available from v1.8 of MultiQC. At the time of writing this is not yet released, so we will install the development version directly from GitHub.

In [ ]:
%pip install --force-reinstall --upgrade git+https://github.com/ewels/MultiQC.git

> NB: You will probably need to restart the notebook kernal after installing MultiQC

Now let's import the `multiqc` package into your workbook:

In [ ]:
import multiqc

Great! Now let's check that it's working properly by printing the version that we're using:

In [ ]:
print(multiqc.__version__)

Before we can use any outputs from MultiQC, we must first run it on some data. The [MultiQC website](https://multiqc.info/) has the logs used for all of the example reports on the homepage available for download, so let's grab the files for the RNA-seq report.

In [ ]:
!wget https://multiqc.info/examples/rna-seq/data.zip
!unzip -o data.zip
!rm data.zip

You should now see a folder called `data/` in your notebook work directory with a bunch of log files within from a typical RNA-seq analysis run.

Now let's run MultiQC on those files.

In [ ]:
multiqc.run('./data/')

Ok great! Now we have a report, we can show it inside the notebook.

> Note that we can use either `IFrame` or not `HTML`. Google Colab doesn't like to render files in iframes and HTML embedding seems to work fine. When running Jupyter Lab locally the notebook CSS and JavaScript interacts with the report and breaks stuff though, so IFrame is better if you can use it.

In [ ]:
import IPython

# Best if using Google Colab
IPython.display.HTML(filename='./multiqc_report.html')

# Best if running locally
# IPython.display.IFrame('./multiqc_report.html', '100%', 600)